In [ ]:
pip install openai

In [ ]:
pip install langchain

In [ ]:
pip install langchain_community

In [ ]:
pip install faiss-cpu

In [ ]:
!pip install langchain_groq

In [ ]:
pip install pypdf

# Importing Libraries

In [18]:
GROQ_API_KEY="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"

In [31]:
import os
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain.chains import ConversationalRetrievalChain

# 1. Set GROQ API key
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

# 2. Load PDF and split into chunks
loader = PyPDFLoader("Java Notes.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

# 3. Embed chunks and store in FAISS index
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(splits, embedding_model)

# 4. Setup Groq Chat model with LLaMA3
llm = ChatGroq(temperature=0, model_name="llama3-8b-8192")


In [ ]:
# 5. Condense question prompt (used for follow-up questions)
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""
Given the following conversation and a follow-up question, rephrase the follow-up to be a standalone question.

Chat History:
{chat_history}
Follow-up Input: {question}
Standalone question:
""")

#6. Create the conversational retrieval chain
qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
    return_source_documents=True,
    verbose=False
)


In [35]:
#7. Start chat loop
chat_history = []

query = input("\nYou: ")
result = qa.invoke({"question": query, "chat_history": chat_history})
print("\nBot:", result["answer"])
chat_history.append((query, result["answer"]))


You: what is oops in java

Bot: I can answer that!

Object-Oriented Programming (OOP) is a programming paradigm that revolves around the concept of "objects" and the interactions between them. In Java, OOP is a fundamental concept that allows developers to create reusable and modular code.

The four main principles of OOP in Java are:

1. Encapsulation: This means bundling data and methods that operate on that data within a single unit, called a class. This helps to hide the implementation details from the outside world and provides a way to control access to the data.
2. Abstraction: This involves showing only the necessary information to the outside world while hiding the internal implementation details. In Java, abstraction is achieved through abstract classes and interfaces.
3. Inheritance: This allows a class to inherit the properties and behavior of another class, known as the superclass or parent class. In Java, inheritance is used to create a hierarchy of classes.
4. Polymorph